In [1]:
!pip install transformers
import tensorflow as tf
import numpy as np
import pandas as pd
import sklearn
import torch
from sklearn import metrics
import transformers
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import json

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 94.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.6 MB/s eta 0:00:00


## Set up TPU and prepare data:

In [2]:
# Using the template in: News classification: fine-tuning RoBERTa on TPUs with TensorFlow:
## https://towardsdatascience.com/news-category-classification-fine-tuning-roberta-on-tpus-with-tensorflow-f057c37b093


In [3]:
# Detect and initialize the TPU, as done in article above:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

batch_size=32 * tpu_strategy.num_replicas_in_sync
print('Batch size:', batch_size)


Batch size: 256


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# LOAD BOOKING77-OOS:

dataset_path = "drive/My Drive/Colab Notebooks/BANKING77-OOS/"
!ls "drive/My Drive/Colab Notebooks/BANKING77-OOS/"

test  train  val


In [6]:
# Create a dataframe with all training, testing and validation data in it:
# Later on, a train-val divide will occur:
seq_data = []
label_data = []

for filetype in ['test', 'val', 'train']:

    with open(dataset_path + filetype + "/seq.in", 'r', newline="\n") as seq_file:
        for line in seq_file:
          seq_data.append(line.strip())

    with open(dataset_path + filetype + "/label.txt", 'r', newline="\n") as label_file:
        for line in label_file:
          label_data.append(line.strip())

    dataset = pd.DataFrame(list(zip(seq_data, label_data)),
               columns =['utterance', 'label'])


In [7]:
labels = list(dataset.label)
categories=sorted(list(set(labels)))
n_categories=len(categories)

In [8]:
# Number of categories/classes:
len(categories)

50

In [9]:
# Training data summary:
for cat in categories:
    print("Category: {}, # utterances: {}".format(cat,len(dataset[dataset.label==cat])))

print("Total utterances: {}".format(len(dataset.label)))

Category: Refund_not_showing_up, # utterances: 202
Category: activate_my_card, # utterances: 199
Category: apple_pay_or_google_pay, # utterances: 166
Category: automatic_top_up, # utterances: 167
Category: balance_not_updated_after_bank_transfer, # utterances: 211
Category: balance_not_updated_after_cheque_or_cash_deposit, # utterances: 221
Category: beneficiary_not_allowed, # utterances: 196
Category: cancel_transfer, # utterances: 197
Category: card_about_to_expire, # utterances: 169
Category: card_arrival, # utterances: 193
Category: card_linking, # utterances: 179
Category: card_payment_fee_charged, # utterances: 227
Category: card_payment_not_recognised, # utterances: 208
Category: card_payment_wrong_exchange_rate, # utterances: 207
Category: cash_withdrawal_charge, # utterances: 217
Category: cash_withdrawal_not_recognised, # utterances: 200
Category: change_pin, # utterances: 162
Category: country_support, # utterances: 169
Category: declined_card_payment, # utterances: 193
Cate

In [10]:
# Create a label-to-index map:
label_to_ix = {}
for cat in categories:
    for word in cat.split():
        if word not in label_to_ix:
            label_to_ix[word]=len(label_to_ix)
label_to_ix

{'Refund_not_showing_up': 0,
 'activate_my_card': 1,
 'apple_pay_or_google_pay': 2,
 'automatic_top_up': 3,
 'balance_not_updated_after_bank_transfer': 4,
 'balance_not_updated_after_cheque_or_cash_deposit': 5,
 'beneficiary_not_allowed': 6,
 'cancel_transfer': 7,
 'card_about_to_expire': 8,
 'card_arrival': 9,
 'card_linking': 10,
 'card_payment_fee_charged': 11,
 'card_payment_not_recognised': 12,
 'card_payment_wrong_exchange_rate': 13,
 'cash_withdrawal_charge': 14,
 'cash_withdrawal_not_recognised': 15,
 'change_pin': 16,
 'country_support': 17,
 'declined_card_payment': 18,
 'declined_cash_withdrawal': 19,
 'declined_transfer': 20,
 'direct_debit_payment_not_recognised': 21,
 'disposable_card_limits': 22,
 'edit_personal_details': 23,
 'exchange_charge': 24,
 'exchange_via_app': 25,
 'extra_charge_on_statement': 26,
 'failed_transfer': 27,
 'fiat_currency_support': 28,
 'getting_spare_card': 29,
 'lost_or_stolen_phone': 30,
 'order_physical_card': 31,
 'pending_card_payment': 32,

In [11]:
def indicize_labels(labels):
    """Transforms categorical labels into indices"""
    indices=[]
    for j in range(len(labels)):
      indices.append(label_to_ix[labels[j]])
    return indices

In [12]:
n_elements = len(indicize_labels(labels))

In [13]:
indices = indicize_labels(labels)
texts = list(dataset.utterance)

In [14]:
# View example:
texts[1345]

'i am travelling for two more weeks but really need to change my pin asap. what do i do?'

In [15]:
labels[1345]

'change_pin'

In [16]:
label_to_ix[labels[1345]]

16

In [17]:
indices[1345]

16

In [18]:
# Tokenize texts:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
#tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='tf')


In [19]:
# Create tensorflow datasets with a 10% split for validation:
dataset=tf.data.Dataset.from_tensor_slices((dict(inputs), indices))

val_size=int(0.1*n_elements)
val_ds=dataset.take(val_size).batch(batch_size, drop_remainder=True)
train_ds=dataset.skip(val_size).batch(batch_size, drop_remainder=True)

## Build training loop using RoBERTa base model (with TPU):

In [20]:
with tpu_strategy.scope():
  model_name = "roberta-base"
  model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=n_categories)
  #model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=n_categories)
  model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5, clipnorm=1.),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.metrics.SparseCategoricalAccuracy(),tf.keras.metrics.SparseTopKCategoricalAccuracy(k=3, name='Sparse_Top_3_Categorical_Accuracy')],
        )

model.fit(train_ds, validation_data=val_ds, epochs=60, verbose=1)

model.save_weights(f'./saved_weights_{model_name}_booking77-oos.h5')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

Epoch 1/60
33/33 [==============================] - 104s 452ms/step - loss: 3.9221 - sparse_categorical_accuracy: 0.0174 - Sparse_Top_3_Categorical_Accuracy: 0.0537 - val_loss: 3.9364 - val_sparse_categorical_accuracy: 0.0000e+00 - val_Sparse_Top_3_Categorical_Accuracy: 0.0143
Epoch 2/60
33/33 [==============================] - 7s 217ms/step - loss: 3.9135 - sparse_categorical_accuracy: 0.0224 - Sparse_Top_3_Categorical_Accuracy: 0.0669 - val_loss: 3.9288 - val_sparse_categorical_accuracy: 0.0052 - val_Sparse_Top_3_Categorical_Accuracy: 0.0404
Epoch 3/60
33/33 [==============================] - 7s 218ms/step - loss: 3.8809 - sparse_categorical_accuracy: 0.0284 - Sparse_Top_3_Categorical_Accuracy: 0.0987 - val_loss: 3.8158 - val_sparse_categorical_accuracy: 0.0938 - val_Sparse_Top_3_Categorical_Accuracy: 0.1784
Epoch 4/60
33/33 [==============================] - 7s 216ms/step - loss: 3.6961 - sparse_categorical_accuracy: 0.1445 - Sparse_Top_3_Categorical_Accuracy: 0.3028 - val_loss: 3.5

In [21]:
# Quick grid/hyperparameter search: Consider early stopping on val loss instead, as an option along with lr decay/scheduler:
## roberta-base:

# lr=3e-5 and 15 epochs reached train acc: 0.9216, val acc: 0.9049
# lr=3e-5 and 30 epochs reached train acc: 0.9732, val acc: 0.9023 [overfit]
# lr=3e-5 and 20 epochs reached train acc: 0.9464, val acc: 0.9036
# lr=3e-5 and 25 epochs reached train acc: 0.9685, val acc: 0.9180

# lr=2e-5 and 50 epochs reached 0.9902 acc, val acc: 0.9154 [overfit]
# lr=2e-5 and 30 epochs reached 0.9607 acc, val acc: 0.9076

# lr=1e-5 and 30 epochs reached 0.9206 acc, val acc: 0.9076
# lr=1e-5 and 50 epochs reached 0.9615 acc, val acc: 0.9062
# lr=1e-5 and 60 epochs reached train acc: 0.9772 , val acc: 0.9102


# bert base uncased :
## 1e-5 and 60 epochs reached 0.9693 acc

## Test model/ Extract predictions from model:

In [22]:
# The final model reached a 0.9102 top-1-accuracy on validation set, and a 0.9740 top-3-accuracy on the validation set.
# To be more exact, one should actually split the data into train, test and validation and finally evaluate on the test dataset, instead.

In [23]:
def get_pred(msg):

  with torch.no_grad():

    inputs = tokenizer(msg, padding=True, truncation=True, return_tensors='tf')
    output = model(inputs)[0]
    pred_label = np.argmax(output, axis=1)[0]

    prediction=list(label_to_ix.keys())[pred_label]
  return prediction

In [24]:
get_pred('i am travelling for two more weeks but really need to change my pin asap. what do i do?')

'change_pin'

In [25]:
get_pred("How do I go about topping up with Google Pay?")

'apple_pay_or_google_pay'

In [26]:
get_pred("I want a refund")

'request_refund'

In [27]:
get_pred("I topped up but it did not go through")

'top_up_failed'

In [28]:
get_pred("I topped up and it is still processing. why? ")

'pending_top_up'

In [29]:
# Response with typos:
get_pred("I toppt up and it still processin.....  ")

'pending_top_up'